# Guía 6: Sumarización Aplicada con Phi-3 y Gradio

## Descripción

En esta guía vas a construir una aplicación web interactiva para sumarizar textos usando Phi-3 y la biblioteca Gradio. A diferencia de la Guía 4 donde usamos modelos especializados en sumarización, acá vamos a usar un LLM de propósito general (Phi-3) con prompt engineering para generar resúmenes de alta calidad.

Al finalizar esta guía vas a poder:
- Implementar sumarización con LLMs instruction-tuned
- Crear interfaces web interactivas con Gradio
- Comparar diferentes enfoques de sumarización (modelos específicos vs LLMs generales)
- Aplicar prompt engineering para mejorar la calidad de resúmenes
- Compartir aplicaciones de NLP con otros usuarios sin conocimientos técnicos

---

## Configuración del Entorno

### Requisitos

Para esta guía necesitás:

1. **Google Colab con GPU activada:**
   - Andá a `Entorno de ejecución > Cambiar tipo de entorno de ejecución`
   - Seleccioná `Acelerador de hardware > GPU`
   - Elegí `Tipo de GPU > T4`

2. **Conexión a Internet** para:
   - Descargar Phi-3 (~2GB)
   - Crear el túnel público de Gradio

La GPU es necesaria para ejecutar Phi-3 con velocidad razonable.

## Instalación de Dependencias

In [1]:
%%capture
# Instalamos las bibliotecas necesarias
# gradio: Para crear la interfaz web interactiva
# transformers: Para usar Phi-3
!pip install gradio -q
!pip install transformers accelerate

### Bibliotecas Instaladas

**Gradio**: Framework para crear demos interactivos de ML
- Genera interfaces web automáticamente desde funciones Python
- Soporta compartir aplicaciones públicamente
- Maneja diferentes tipos de entrada/salida (texto, imagen, audio, etc.)

**Transformers**: Ya lo conocemos de guías anteriores
- Acceso a Phi-3 y otros modelos

**Accelerate**: Optimiza la ejecución de modelos grandes

## Marco Teórico

### Sumarización con LLMs

Existen dos enfoques principales para sumarización con modelos Transformer:

#### 1. Modelos Especializados (Encoder-Decoder)

Modelos como mT5, BART, Pegasus fueron específicamente entrenados para sumarización:
- **Ventaja**: Optimizados para esta tarea, rápidos
- **Limitación**: Menos flexibles, resúmenes de longitud/estilo fijo

#### 2. LLMs de Propósito General (Decoder-Only)

Modelos como GPT, Phi-3, LLaMA pueden resumir mediante prompts:
- **Ventaja**: Muy flexibles, podés especificar estilo, longitud, enfoque
- **Ventaja**: Entienden instrucciones complejas
- **Limitación**: Más lentos y requieren más memoria

### ¿Cuándo usar cada enfoque?

**Modelos especializados** (mT5, BART):
- Resúmenes simples de una oración
- Procesamiento de grandes volúmenes
- Latencia crítica
- Recursos computacionales limitados

**LLMs generales** (Phi-3, GPT):
- Resúmenes con requisitos específicos ("resumen ejecutivo", "resumen para niños")
- Múltiples estilos de resumen
- Extracción de información específica
- Resúmenes con análisis ("resumí y destacá los puntos clave")

### Prompt Engineering para Sumarización

La calidad del resumen depende fuertemente del prompt. Elementos clave:

1. **Instrucción clara**: "Resumí el siguiente texto"
2. **Especificaciones**: Longitud, estilo, audiencia
3. **Formato**: Párrafo, lista de puntos, etc.
4. **Contexto**: Propósito del resumen

Ejemplo de prompt efectivo:
```
Resume el siguiente texto en un párrafo de 3-5 oraciones,
destacando las ideas principales y conclusiones más importantes.

Texto: [texto aquí]

Resumen:
```

### ¿Qué es Gradio?

**Gradio** es una biblioteca Python que convierte funciones en aplicaciones web interactivas automáticamente.

Ventajas:
- **Cero código frontend**: No necesitás HTML/CSS/JavaScript
- **Compartible**: Genera URL pública para compartir con otros
- **Integración simple**: Funciona con cualquier función Python
- **Componentes ricos**: Texto, archivos, imágenes, audio, video, etc.

Casos de uso:
- Demos de modelos de ML
- Prototipos rápidos
- Herramientas internas de equipo
- Validación de modelos con usuarios no técnicos

## Parte 1: Carga del Modelo Phi-3

Primero cargamos el modelo que usaremos para sumarización.

In [2]:
import gradio as gr
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

print("Bibliotecas importadas correctamente")

Bibliotecas importadas correctamente


In [3]:
# Cargamos Phi-3
# Este proceso puede tardar 1-2 minutos la primera vez
model_id = "microsoft/Phi-3-mini-4k-instruct"

print("Cargando Phi-3... (esto puede tardar un momento)")

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",       # device_map: "auto" distribuye el modelo entre GPU/CPU automáticamente
    torch_dtype="auto",      # torch_dtype: Usa precisión óptima automáticamente
    trust_remote_code=False, # trust_remote_code: False usa código nativo de transformers (más estable)
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

print("Modelo Phi-3 cargado exitosamente")
print(f"Parámetros: 3.8B")
print(f"Contexto máximo: 4096 tokens")

Cargando Phi-3... (esto puede tardar un momento)


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

Modelo Phi-3 cargado exitosamente
Parámetros: 3.8B
Contexto máximo: 4096 tokens


### Configuración del Modelo

**device_map="auto"**:
- Detecta automáticamente los dispositivos disponibles (GPU, CPU)
- Distribuye capas del modelo para optimizar uso de memoria
- Si tenés GPU, carga todo allí; si no alcanza la memoria, divide entre GPU y CPU

**torch_dtype="auto"**:
- Selecciona automáticamente la precisión numérica
- En GPU con soporte: usa float16 (más rápido, menos memoria)
- En CPU: usa float32 (más preciso)

**trust_remote_code=False**:
- Usa la implementación nativa de transformers
- Phi-3 está integrado en transformers desde la versión 4.41.2
- Más estable y mejor mantenido que el código personalizado
- Solo activar trust_remote_code para modelos que no estén integrados en transformers

In [4]:
# Creamos un pipeline para facilitar la generación
generador = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

print("Pipeline de generación creado")

Device set to use cuda:0


Pipeline de generación creado


## Parte 2: Función de Sumarización

Ahora implementamos la lógica de sumarización usando prompt engineering.

In [5]:
def resumir_con_phi3(texto, longitud="medio"):
    """
    Resume un texto usando Phi-3 con prompt engineering.

    Parámetros:
    - texto: El texto a resumir
    - longitud: "corto" (1-2 oraciones), "medio" (3-5 oraciones), "largo" (párrafo completo)

    Returns:
    - Resumen generado por Phi-3
    """
    # Definimos instrucciones según la longitud deseada
    instrucciones = {
        "corto": "Resume el siguiente texto en 1-2 oraciones capturando solo la idea principal.",
        "medio": "Resume el siguiente texto en un párrafo de 3-5 oraciones, destacando las ideas principales.",
        "largo": "Resume el siguiente texto en un párrafo detallado, manteniendo los puntos clave y conclusiones."
    }

    instruccion = instrucciones.get(longitud, instrucciones["medio"])

    # Construimos el prompt usando chat template de Phi-3
    # Phi-3 funciona mejor con formato de conversación estructurado
    messages = [
        {"role": "system", "content": "Eres un asistente experto en resumir textos en español de forma clara y concisa."},
        {"role": "user", "content": f"{instruccion}\n\nTexto:\n{texto}\n\nPor favor, proporciona solo el resumen sin introducción ni explicaciones adicionales."}
    ]

    # Aplicamos el chat template del tokenizer
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    # Generamos el resumen
    resultado = generador(
        prompt,
        max_new_tokens=200,      # max_new_tokens: Máximo de tokens a generar (controla longitud)
        do_sample=True,          # do_sample: True para permitir temperature (más variado)
        temperature=0.7,         # temperature: Controla creatividad (0.7 = balance)
        top_p=0.9,              # top_p: Nucleus sampling para mejor calidad
        return_full_text=False   # return_full_text: Solo devuelve el resumen, no el prompt
    )

    # Extraemos el resumen de la respuesta
    resumen = resultado[0]['generated_text'].strip()

    # Limpieza: A veces el modelo agrega etiquetas o prefijos
    if resumen.startswith("Resumen:"):
        resumen = resumen[8:].strip()

    # Remover posibles tags de chat que puedan quedar
    resumen = resumen.replace("<|assistant|>", "").replace("<|end|>", "").strip()

    return resumen

print("Función de sumarización definida")

Función de sumarización definida


### Análisis de la Función

**Prompt Engineering:**
- Instrucción clara al inicio
- Delimitadores explícitos ("Texto:", "Resumen:")
- Especificación de longitud en lenguaje natural

**Chat Template de Phi-3:**
- Usa `tokenizer.apply_chat_template()` para formato correcto
- Estructura de mensajes con roles: system, user, assistant
- Phi-3 está entrenado específicamente para este formato
- Mejora significativamente la calidad de respuestas

**Parámetros de generación:**

**max_new_tokens=200**:
- Limita la longitud del resumen
- 200 tokens equivalen aproximadamente a 150 palabras
- Ajustable según necesidades

**do_sample=True**:
- Habilita sampling estocástico para variedad controlada
- Permite que temperature tenga efecto
- Genera resúmenes más variados y naturales

**temperature=0.7**:
- Controla la distribución de probabilidad de tokens
- Balance entre creatividad y coherencia
- Valores más bajos (0.3-0.5): más conservador
- Valores más altos (0.8-1.0): más variado

**top_p=0.9**:
- Nucleus sampling: considera tokens que suman 90% de probabilidad
- Mejora calidad evitando tokens de muy baja probabilidad
- Complementa bien con temperature

**return_full_text=False**:
- Solo devuelve el texto generado
- No incluye el prompt en la respuesta

**Limpieza del output**:
- Remueve prefijos comunes ("Resumen:")
- Elimina tags de chat que puedan filtrarse
- Garantiza output limpio para el usuario

### Alternativa: Generación Determinística

Si preferís resultados 100% consistentes (mismo texto produce mismo resumen):

```python
resultado = generador(
    prompt,
    max_new_tokens=200,
    do_sample=False,        # Greedy decoding
    return_full_text=False
)
```

**Nota**: Cuando do_sample=False, no usar temperature ni top_p.

**Trade-offs**:
- do_sample=True: Más variado, natural, pero menos predecible
- do_sample=False: Determinístico, predecible, pero puede ser repetitivo

## Parte 3: Probando la Función

Antes de crear la interfaz, probemos la función con un texto de ejemplo.

In [6]:
# Texto de prueba: Biografía de Ludwig Wittgenstein
texto_ejemplo = """Ludwig Josef Johann Wittgenstein (Viena, 26 de abril de 1889-Cambridge, 29 de abril de 1951),
conocido como Ludwig Wittgenstein, fue un filósofo, matemático, lingüista y lógico austríaco,
posteriormente nacionalizado británico. Su primera teoría plantea que existe una relación biunívoca entre las palabras y las cosas,
y que las proposiciones que encadenan las palabras constituyen «imágenes» de la realidad (Tractatus logico-philosophicus),
que influyó en gran medida a los positivistas lógicos del Círculo de Viena, movimiento del que nunca se consideró miembro.
Tiempo después, el Tractatus fue severamente criticado por el propio Wittgenstein en Los cuadernos azul y marrón y en sus Investigaciones filosóficas,
así como en Observaciones filosóficas, obras póstumas en beneficio de una concepción más restringida y concreta,
calificada de «juego de lenguaje», en la que destaca el aspecto humano del lenguaje, es decir, su imprecisión y variabilidad
según las situaciones. Fue discípulo de Bertrand Russell en el Trinity College de la Universidad de Cambridge,
donde más tarde también él llegó a ser profesor."""

print("Texto original:")
print("="*80)
print(texto_ejemplo)
print("\n" + "="*80)

Texto original:
Ludwig Josef Johann Wittgenstein (Viena, 26 de abril de 1889-Cambridge, 29 de abril de 1951),
conocido como Ludwig Wittgenstein, fue un filósofo, matemático, lingüista y lógico austríaco,
posteriormente nacionalizado británico. Su primera teoría plantea que existe una relación biunívoca entre las palabras y las cosas,
y que las proposiciones que encadenan las palabras constituyen «imágenes» de la realidad (Tractatus logico-philosophicus),
que influyó en gran medida a los positivistas lógicos del Círculo de Viena, movimiento del que nunca se consideró miembro.
Tiempo después, el Tractatus fue severamente criticado por el propio Wittgenstein en Los cuadernos azul y marrón y en sus Investigaciones filosóficas,
así como en Observaciones filosóficas, obras póstumas en beneficio de una concepción más restringida y concreta,
calificada de «juego de lenguaje», en la que destaca el aspecto humano del lenguaje, es decir, su imprecisión y variabilidad
según las situaciones. Fue di

In [7]:
# Resumen corto (1-2 oraciones)
print("\nRESUMEN CORTO:")
print("-"*80)
resumen_corto = resumir_con_phi3(texto_ejemplo, longitud="corto")
print(resumen_corto)


RESUMEN CORTO:
--------------------------------------------------------------------------------
Ludwig Wittgenstein fue un filósofo austriaco-británico que inicialmente propuso una relación biunívoca entre las palabras y las cosas en su obra "Tractatus logico-philosophicus", influenciando al Círculo de Viena, pero luego criticó su propio trabajo y desarrolló una concepción más restringida y humana del lenguaje, enfatizando su imprecisión y variabilidad.


In [8]:
# Resumen medio (3-5 oraciones)
print("\nRESUMEN MEDIO:")
print("-"*80)
resumen_medio = resumir_con_phi3(texto_ejemplo, longitud="medio")
print(resumen_medio)


RESUMEN MEDIO:
--------------------------------------------------------------------------------
Ludwig Wittgenstein fue un filósofo y matemático que desarrolló teorías significativas sobre lenguaje y realidad, inicialmente con el Tractatus logico-philosophicus que sugería una relación directa entre palabras y objetos, formando imágenes de la realidad. Posteriormente, Wittgenstein cambió su perspectiva en su obra posterior, Los cuadernos azul y marrón, Observaciones filosóficas, y Investigaciones filosóficas, donde promovió una visión más humana del lenguaje enfatizando su imprecisión y variabilidad. A pesar de no haber sido miembro del Círculo de Viena, su trabajo influyó en los positivistas lógicos de dicho movimiento. Finalmente, Wittgenstein fue discípulo de Bertrand Russell en Cambridge


In [9]:
# Resumen largo (párrafo completo)
print("\nRESUMEN LARGO:")
print("-"*80)
resumen_largo = resumir_con_phi3(texto_ejemplo, longitud="largo")
print(resumen_largo)


RESUMEN LARGO:
--------------------------------------------------------------------------------
Ludwig Wittgenstein fue un prominente filósofo y matemático, que inicialmente desarrolló la teoría de que las palabras tienen una relación biunívoca con las cosas y que las proposiciones forman imágenes de la realidad, según su obra "Tractatus logico-philosophicus". Esta teoría influyó en los positivistas lógicos del Círculo de Viena, a pesar de que Wittgenstein nunca formó parte del grupo. Posteriormente, Wittgenstein cambió su enfoque en su libro "Los cuadernos azul y marrón" y sus "Investigaciones filosóficas", criticando su propia teoría anterior y promoviendo una visión más restrictiva y concreta del lenguaje como un "juego de lenguaje". Esta nueva perspectiva resaltaba la natur


### Evaluación de Resúmenes

Al comparar los resúmenes, notá:

1. **Fidelidad**: ¿El resumen captura las ideas principales?
2. **Concisión**: ¿Elimina información redundante?
3. **Coherencia**: ¿Es gramaticalmente correcto y fluido?
4. **Completitud**: ¿Incluye todos los puntos importantes?

Los resúmenes generados por Phi-3 tienden a ser:
- **Abstractivos**: Generan nuevas formulaciones
- **Informativos**: Capturan la esencia del texto
- **Flexibles**: Se adaptan a las instrucciones del prompt

## Parte 4: Creando la Interfaz con Gradio

Ahora vamos a crear una aplicación web interactiva que cualquiera puede usar, incluso sin conocimientos de programación.

In [10]:
# Definimos la función que se conectará a la interfaz
def resumir_texto_interfaz(texto, longitud):
    """
    Función wrapper para la interfaz Gradio.
    Incluye validaciones y manejo de errores.
    """
    # Validación: Texto no vacío
    if not texto or len(texto.strip()) < 50:
        return "Error: El texto debe tener al menos 50 caracteres para resumir."

    # Validación: Texto no excesivamente largo
    if len(texto.split()) > 1000:
        return "Error: El texto es demasiado largo (máximo 1000 palabras). Dividí el texto en partes más pequeñas."

    try:
        # Generar resumen
        resumen = resumir_con_phi3(texto, longitud=longitud)
        return resumen
    except Exception as e:
        return f"Error al generar resumen: {str(e)}"

print("Función de interfaz definida")

Función de interfaz definida


### Validaciones Importantes

La función incluye validaciones para:

1. **Texto mínimo**: Al menos 50 caracteres
   - Textos muy cortos no necesitan resumen
   - Evita errores del modelo

2. **Texto máximo**: Máximo 1000 palabras
   - Protege contra timeouts
   - Evita exceder el contexto del modelo

3. **Manejo de excepciones**: Try/except para errores inesperados
   - Muestra mensajes amigables al usuario
   - Evita que la aplicación se caiga

In [11]:
# Creamos la interfaz Gradio
interfaz = gr.Interface(
    fn=resumir_texto_interfaz,           # fn: La función a ejecutar

    inputs=[
        gr.Textbox(
            label="Texto a resumir",      # label: Etiqueta que ve el usuario
            placeholder="Pegá aquí el texto que querés resumir...",  # placeholder: Texto de ayuda
            lines=10,                     # lines: Altura del cuadro de texto (en líneas)
        ),
        gr.Radio(
            choices=["corto", "medio", "largo"],  # choices: Opciones disponibles
            label="Longitud del resumen",         # label: Título del control
            value="medio"                         # value: Valor por defecto
        )
    ],

    outputs=gr.Textbox(
        label="Resumen generado",           # label: Título del resultado
        lines=8                            # lines: Altura del cuadro de resultado
    ),

    title="Resumidor de Textos con Phi-3",    # title: Título de la aplicación

    description="""Esta aplicación usa el modelo Phi-3 de Microsoft para generar resúmenes de textos en español.

    Instrucciones:
    1. Pegá o escribí el texto que querés resumir (entre 50 y 1000 palabras)
    2. Elegí la longitud del resumen deseada
    3. Hacé clic en 'Submit' y esperá unos segundos

    El modelo genera resúmenes abstractivos que capturan las ideas principales del texto original.""",  # description: Descripción de la aplicación

    examples=[
        [
            "La inteligencia artificial está transformando múltiples industrias. En la medicina, permite diagnósticos más precisos mediante el análisis de imágenes médicas. En el transporte, los vehículos autónomos prometen revolucionar la movilidad urbana. Sin embargo, estos avances plantean importantes desafíos éticos y laborales que la sociedad debe abordar.",
            "medio"
        ],
        [
            texto_ejemplo,
            "corto"
        ]
    ],  # examples: Ejemplos precargados para que el usuario pruebe

    theme="soft"  # theme: Tema visual ("default", "soft", "dark", etc.)
)

print("Interfaz Gradio creada")

Interfaz Gradio creada


### Componentes de la Interfaz

**gr.Interface**:
- Componente principal que crea la aplicación web
- Conecta inputs → función → outputs automáticamente

**Inputs**:
1. **gr.Textbox**: Cuadro de texto para el input
   - `lines`: Controla altura (más líneas = más visible)
   - `placeholder`: Texto de ayuda que desaparece al escribir
   - `label`: Título descriptivo

2. **gr.Radio**: Botones de opción (solo una selección)
   - `choices`: Lista de opciones
   - `value`: Opción seleccionada por defecto
   - Alternativas: `gr.Dropdown` (menú desplegable), `gr.Checkbox` (múltiples opciones)

**Output**:
- **gr.Textbox**: Muestra el resumen generado
- Podría ser también: `gr.Markdown`, `gr.HTML`, `gr.JSON`, etc.

**Metadata**:
- `title`: Nombre de la aplicación
- `description`: Instrucciones y contexto
- `examples`: Casos de uso precargados para facilitar pruebas
- `theme`: Estilo visual de la interfaz

In [ ]:
# Lanzamos la aplicación
# share=True crea una URL pública que podés compartir
# La URL es válida por 72 horas
interfaz.launch(
    share=True,          # share: True para crear URL pública, False para solo local
    debug=True           # debug: Muestra información detallada de errores
)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://e8d2d084a1c6f3773b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


### Parámetros de launch()

**share=True**:
- Crea un túnel público temporal
- Genera URL tipo: `https://xxxxx.gradio.app`
- Válida por 72 horas
- Cualquiera con el link puede usar la aplicación
- Útil para: Demos, compartir con colegas, testing con usuarios

**share=False**:
- Solo accesible localmente
- URL tipo: `http://127.0.0.1:7860`
- Más seguro para desarrollo

**debug=True**:
- Muestra stack traces completos de errores
- Útil durante desarrollo
- Desactivar en producción por seguridad

**Otros parámetros útiles**:
- `server_name`: IP del servidor (default: "127.0.0.1")
- `server_port`: Puerto (default: 7860)
- `auth`: Autenticación con usuario/contraseña
- `inbrowser`: Abre automáticamente en el navegador

## Parte 5: Mejoras y Extensiones

Ahora que tenemos la aplicación básica funcionando, veamos cómo extenderla.

### Extensión 1: Múltiples Estilos de Resumen

Podemos modificar la función para ofrecer diferentes estilos además de longitudes.

In [ ]:
def resumir_texto_avanzado(texto, longitud, estilo):
    """
    Versión avanzada con diferentes estilos de resumen.
    """
    # Plantillas de instrucciones por estilo
    plantillas_estilo = {
        "general": "Resume el siguiente texto de forma clara y concisa.",
        "ejecutivo": "Genera un resumen ejecutivo del siguiente texto, destacando los puntos clave para toma de decisiones.",
        "académico": "Crea un resumen académico del siguiente texto, manteniendo la terminología técnica y estructura formal.",
        "simple": "Resume el siguiente texto usando lenguaje simple y accesible, como si explicaras a alguien sin conocimientos previos del tema."
    }

    # Especificaciones de longitud
    especificaciones_longitud = {
        "corto": "en 1-2 oraciones",
        "medio": "en un párrafo de 3-5 oraciones",
        "largo": "en un párrafo detallado"
    }

    # Construir prompt combinando estilo y longitud con chat template
    instruccion_base = plantillas_estilo.get(estilo, plantillas_estilo["general"])
    especificacion = especificaciones_longitud.get(longitud, especificaciones_longitud["medio"])

    messages = [
        {"role": "system", "content": "Eres un asistente experto en resumir textos en español."},
        {"role": "user", "content": f"{instruccion_base} Resume {especificacion}.\n\nTexto:\n{texto}"}
    ]

    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    # Generar
    resultado = generador(
        prompt,
        max_new_tokens=250,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        return_full_text=False
    )

    resumen = resultado[0]['generated_text'].strip()

    # Limpieza
    if "Resumen:" in resumen:
        resumen = resumen.split("Resumen:")[-1].strip()

    resumen = resumen.replace("<|assistant|>", "").replace("<|end|>", "").strip()

    return resumen

print("Función avanzada definida")

### Extensión 2: Procesamiento por Lotes

Para resumir múltiples textos de una vez.

In [ ]:
def resumir_lote(archivo_texto, longitud):
    """
    Resume múltiples textos desde un archivo.

    Formato del archivo:
    Texto 1
    ---
    Texto 2
    ---
    Texto 3
    """
    # Leer archivo
    with open(archivo_texto.name, 'r', encoding='utf-8') as f:
        contenido = f.read()

    # Dividir textos por separador
    textos = contenido.split('---')

    # Resumir cada uno
    resumenes = []
    for i, texto in enumerate(textos, 1):
        texto = texto.strip()
        if len(texto) > 50:
            resumen = resumir_con_phi3(texto, longitud=longitud)
            resumenes.append(f"Texto {i}:\n{resumen}\n")

    return "\n---\n".join(resumenes)

print("Función de lote definida")

### Extensión 3: Comparación con Otro Modelo

Comparar resúmenes de Phi-3 vs mT5 side-by-side.

In [ ]:
def comparar_modelos(texto, longitud):
    """
    Genera resúmenes con dos modelos diferentes para comparación.
    """
    # Resumen con Phi-3 (ya tenemos la función)
    resumen_phi3 = resumir_con_phi3(texto, longitud=longitud)

    # Resumen con mT5 (de la Guía 4)
    from transformers import pipeline as hf_pipeline
    summarizer_mt5 = hf_pipeline(
        "summarization",
        model="csebuetnlp/mT5_multilingual_XLSum"
    )

    max_lengths = {"corto": 50, "medio": 100, "largo": 150}
    resumen_mt5 = summarizer_mt5(
        texto,
        max_length=max_lengths[longitud],
        min_length=20
    )[0]['summary_text']

    # Formatear comparación
    comparacion = f"""MODELO: Phi-3 (LLM instruction-tuned)
    {resumen_phi3}

    ---

    MODELO: mT5 (Modelo especializado en sumarización)
    {resumen_mt5}
    """

    return comparacion

print("Función de comparación definida")

## Ejercicio: Creá Tu Propia Aplicación

Usá el template siguiente para crear una variante personalizada:

In [ ]:
# Tu turno: Personalizá esta aplicación
def mi_funcion_resumir(texto, parametro1, parametro2):
    # Tu lógica aquí
    pass

mi_interfaz = gr.Interface(
    fn=mi_funcion_resumir,
    inputs=[
        # Define tus inputs aquí
    ],
    outputs=[
        # Define tus outputs aquí
    ],
    title="Mi Aplicación Personalizada",
    description="Tu descripción aquí"
)

# mi_interfaz.launch(share=True)

## Resumen de Conceptos Clave

### Sumarización con LLMs

1. **Dos enfoques**:
   - Modelos especializados (mT5, BART): Rápidos, optimizados, menos flexibles
   - LLMs generales (Phi-3, GPT): Flexibles, controlables via prompts, más lentos

2. **Prompt Engineering**:
   - Instrucción clara
   - Especificación de longitud y estilo
   - Delimitadores explícitos

3. **Parámetros de generación**:
   - `max_new_tokens`: Controla longitud máxima
   - `temperature`: Balance creatividad/coherencia
   - `do_sample`: Determinístico vs estocástico

### Gradio

1. **Componentes básicos**:
   - `gr.Interface`: Crea la aplicación automáticamente
   - `gr.Textbox`: Entrada/salida de texto
   - `gr.Radio`, `gr.Dropdown`: Selección de opciones

2. **Funcionalidades**:
   - `share=True`: URL pública temporal
   - `examples`: Casos de uso precargados
   - `theme`: Personalización visual

3. **Casos de uso**:
   - Demos de modelos ML
   - Prototipos rápidos
   - Herramientas internas
   - Validación con usuarios no técnicos

### Aplicación Práctica

1. **Construcción modular**: Función → Interfaz → Deploy
2. **Validaciones**: Input checking para robustez
3. **Extensibilidad**: Fácil agregar features
4. **Compartible**: URL pública para demos

## Guía de Preguntas y Respuestas

### Preguntas de Comprensión

**1. ¿Cuál es la ventaja principal de usar un LLM general (como Phi-3) para sumarización en lugar de un modelo especializado (como mT5)?**

<details>
<summary>Ver respuesta</summary>

La ventaja principal de LLMs generales es la **flexibilidad** controlada mediante prompt engineering:

**Con LLMs (Phi-3, GPT):**
```python
# Diferentes estilos con el MISMO modelo
"Resume como resumen ejecutivo para CEO"
"Resume en lenguaje simple para un niño"
"Resume destacando solo aspectos técnicos"
"Resume en formato de lista de puntos"
```

**Con modelos especializados (mT5):**
- Generan siempre el mismo tipo de resumen
- No entienden instrucciones complejas
- Necesitarías fine-tune para cada estilo diferente

**Trade-offs:**

LLMs son mejores cuando:
- Necesitás múltiples estilos/formatos
- Querés control granular sobre el output
- La calidad es más importante que la velocidad

Modelos especializados son mejores cuando:
- Necesitás procesar miles de documentos (más rápidos)
- Solo un estilo de resumen es suficiente
- Recursos computacionales son limitados
</details>

---

**2. ¿Por qué es importante el prompt engineering en la sumarización con LLMs?**

<details>
<summary>Ver respuesta</summary>

El prompt engineering es crítico porque los LLMs son extremadamente sensibles a cómo formulás las instrucciones.

**Comparación:**

Prompt malo:
```python
"Resume esto:\n[texto]"
```
Resultado: Resumen vago, longitud impredecible, puede no capturar lo importante

Prompt bueno:
```python
"Resume el siguiente texto en 3-5 oraciones,
destacando las conclusiones principales y datos clave.
Usa lenguaje claro y directo.

Texto: [texto]

Resumen:"
```
Resultado: Resumen específico, longitud controlada, enfocado en lo pedido

**Elementos de un buen prompt:**

1. **Instrucción clara**: "Resume", "Sintetiza", "Extrae las ideas principales"
2. **Especificaciones**: Longitud ("en 3 oraciones"), formato ("como lista"), tono ("formal", "simple")
3. **Contexto**: Para quién es el resumen, qué propósito tiene
4. **Delimitadores**: Separar claramente instrucción, texto y área de respuesta
5. **Restricciones**: "No uses lenguaje técnico", "Mantené la objetividad"

**Impacto**:
Un prompt bien diseñado puede mejorar la calidad del resumen en un 50-70% comparado con prompts genéricos.
</details>

---

**3. ¿Qué hace exactamente el parámetro `device_map="auto"` al cargar Phi-3?**

<details>
<summary>Ver respuesta</summary>

`device_map="auto"` implementa **model parallelism** inteligente: distribuye automáticamente el modelo entre los dispositivos disponibles.

**Proceso de decisión:**

1. **Detecta dispositivos**: GPU(s), CPU, disco
2. **Calcula memoria disponible** en cada dispositivo
3. **Analiza tamaño del modelo**: Phi-3 ≈ 7.6 GB en float16
4. **Distribuye capas** optimizando para:
   - Maximizar uso de GPU (más rápido)
   - Minimizar transferencias entre dispositivos
   - Evitar OOM (Out Of Memory)

**Escenarios:**

**Caso 1: GPU con 16GB de VRAM (típico T4)**
```
device_map = {
    "model.embed_tokens": "cuda:0",
    "model.layers.0-31": "cuda:0",
    "lm_head": "cuda:0"
}
```
Todo en GPU → Máxima velocidad

**Caso 2: GPU con 8GB de VRAM (insuficiente)**
```
device_map = {
    "model.embed_tokens": "cuda:0",
    "model.layers.0-20": "cuda:0",
    "model.layers.21-31": "cpu",
    "lm_head": "cpu"
}
```
Híbrido GPU+CPU → Más lento pero funciona

**Caso 3: Sin GPU**
```
device_map = {"all_layers": "cpu"}
```
Todo en CPU → Muy lento pero accesible

**Alternativas:**
- `device_map="cuda"`: Fuerza GPU (falla si no alcanza memoria)
- `device_map="cpu"`: Fuerza CPU (nunca usa GPU)
- `device_map={custom_dict}`: Control manual de cada capa
</details>

---

**4. ¿Qué es Gradio y en qué se diferencia de crear una aplicación web tradicional?**

<details>
<summary>Ver respuesta</summary>

**Gradio** es un framework que convierte funciones Python en aplicaciones web interactivas automáticamente, eliminando la necesidad de desarrollo frontend tradicional.

**Comparación:**

**Aplicación web tradicional:**
```
1. Backend (Python/Flask/Django):
   - Definir rutas
   - Manejar requests/responses
   - Validación de datos
   - APIs REST o GraphQL

2. Frontend (HTML/CSS/JavaScript/React):
   - Crear formularios
   - Estilizar interfaz
   - Manejar eventos
   - Llamadas API asíncronas
   - Renderizar resultados

3. Deployment:
   - Servidor web (Nginx/Apache)
   - Hosting
   - Dominio
   - HTTPS/SSL

Tiempo estimado: Días/semanas
```

**Con Gradio:**
```python
import gradio as gr

def mi_funcion(input):
    return procesar(input)

gr.Interface(
    fn=mi_funcion,
    inputs="text",
    outputs="text"
).launch(share=True)

Tiempo estimado: Minutos
```

**Ventajas de Gradio:**
1. **Cero código frontend**: Genera automáticamente UI
2. **Deploy instantáneo**: `share=True` crea URL pública
3. **Componentes listos**: Textboxes, sliders, file uploads, etc.
4. **Integración Python**: Funciona con cualquier función Python

**Desventajas de Gradio:**
1. **Personalización limitada**: UI predefinida
2. **No escalable**: Para producción con miles de usuarios, necesitás solución tradicional
3. **Funcionalidad básica**: No maneja autenticación compleja, bases de datos, etc.

**Casos de uso ideales para Gradio:**
- Demos de modelos ML
- Prototipos rápidos
- Herramientas internas de equipo
- Proof of concepts
- Validación de modelos con usuarios

**Cuándo usar aplicación web tradicional:**
- Producto comercial
- Alta carga de usuarios
- UI/UX personalizada
- Lógica de negocio compleja
- Integración con sistemas empresariales
</details>

---

**5. ¿Por qué incluimos validaciones (longitud mínima, máxima) en la función de interfaz?**

<details>
<summary>Ver respuesta</summary>

Las validaciones son críticas para robustez y experiencia de usuario. Sin ellas, la aplicación puede:

**Problemas sin validaciones:**

1. **Texto demasiado corto (< 50 caracteres)**:
   - El modelo puede generar output sin sentido
   - "Resume esto: Hola" → El modelo puede repetir el input o generar basura
   - Desperdicio de recursos computacionales

2. **Texto demasiado largo (> límite del modelo)**:
   - **Timeout**: La generación tarda demasiado
   - **OOM**: Se queda sin memoria y crashea
   - **Truncamiento**: El modelo solo procesa los primeros N tokens, perdiendo información
   - Phi-3 tiene límite de 4096 tokens (≈3000 palabras)

3. **Input vacío o inválido**:
   - Errores de tokenización
   - Crashes sin mensajes claros

**Validaciones implementadas:**

```python
# Mínimo
if len(texto) < 50:
    return "Error: Texto muy corto"
# Previene: Outputs sin sentido, desperdicio de recursos

# Máximo
if len(texto.split()) > 1000:
    return "Error: Texto muy largo"
# Previene: Timeouts, OOM, truncamiento

# Try-except
try:
    return resumir(texto)
except Exception as e:
    return f"Error: {e}"
# Previene: Crashes, mensajes crípticos para el usuario
```

**Beneficios:**
- **UX mejorada**: Mensajes de error claros y accionables
- **Robustez**: La aplicación no crashea
- **Eficiencia**: No desperdicia recursos en inputs inválidos
- **Debugging**: Logs claros de qué salió mal

**Mejores prácticas adicionales:**
- Sanitizar input (remover caracteres especiales problemáticos)
- Rate limiting (limitar requests por usuario/IP)
- Logging de errores para monitoreo
- Mensajes de error amigables, no técnicos
</details>

---

### Preguntas de Aplicación

**6. Si querés agregar autenticación a tu aplicación Gradio (solo usuarios autorizados pueden usarla), ¿cómo lo harías?**

<details>
<summary>Ver respuesta</summary>

Gradio soporta autenticación simple mediante el parámetro `auth`:

```python
# Opción 1: Usuario y contraseña único
interfaz.launch(
    share=True,
    auth=("usuario", "contraseña123")  # Tupla (username, password)
)
```

**Opción 2: Múltiples usuarios**
```python
interfaz.launch(
    share=True,
    auth=[("admin", "admin123"), ("usuario", "user123")]  # Lista de tuplas
)
```

**Opción 3: Autenticación personalizada con función**
```python
def verificar_credenciales(username, password):
    # Tu lógica de verificación (ej: consultar base de datos)
    usuarios_validos = {
        "admin": "admin123",
        "usuario1": "pass1",
        "usuario2": "pass2"
    }
    return usuarios_validos.get(username) == password

interfaz.launch(
    share=True,
    auth=verificar_credenciales  # Función que retorna True/False
)
```

**Opción 4: Autenticación avanzada (producción)**
```python
import os
from functools import wraps

# Verificar contra variables de entorno o base de datos
def auth_avanzada(username, password):
    # Verificar contra hash en base de datos
    import hashlib
    password_hash = hashlib.sha256(password.encode()).hexdigest()
    
    # Consultar DB
    usuario_db = obtener_usuario_de_db(username)
    
    if usuario_db and usuario_db['password_hash'] == password_hash:
        return True
    return False

interfaz.launch(
    share=True,
    auth=auth_avanzada
)
```

**Consideraciones de seguridad:**

⚠️ **Para prototipos/demos:**
- Autenticación básica de Gradio es suficiente
- No almacenar contraseñas en código fuente (usar variables de entorno)

⚠️ **Para producción:**
- Gradio auth es básica, no está diseñada para alta seguridad
- Usar sistema de autenticación robusto (OAuth, JWT)
- HTTPS obligatorio
- Considerar alternativas: FastAPI + OAuth2, Django + auth
</details>

---

**7. ¿Cómo modificarías la aplicación para permitir cargar archivos PDF y resumir su contenido?**

<details>
<summary>Ver respuesta</summary>

Necesitás agregar extracción de texto de PDF y modificar los inputs:

```python
import gradio as gr
import PyPDF2
import io

def extraer_texto_pdf(archivo_pdf):
    """
    Extrae texto de un archivo PDF.
    """
    try:
        # Leer PDF
        pdf_reader = PyPDF2.PdfReader(archivo_pdf.name)
        
        # Extraer texto de todas las páginas
        texto_completo = ""
        for pagina in pdf_reader.pages:
            texto_completo += pagina.extract_text() + "\n"
        
        return texto_completo
    except Exception as e:
        return f"Error al leer PDF: {str(e)}"

def resumir_pdf(archivo_pdf, longitud):
    """
    Resume el contenido de un PDF.
    """
    # Validar que se cargó un archivo
    if archivo_pdf is None:
        return "Error: No se cargó ningún archivo"
    
    # Extraer texto
    texto = extraer_texto_pdf(archivo_pdf)
    
    if texto.startswith("Error"):
        return texto
    
    # Validar longitud
    if len(texto.split()) > 2000:
        # Para PDFs muy largos, resumir por secciones
        return "PDF muy largo. Implementar chunking..."
    
    # Resumir
    return resumir_con_phi3(texto, longitud=longitud)

# Interfaz
interfaz_pdf = gr.Interface(
    fn=resumir_pdf,
    inputs=[
        gr.File(
            label="Cargar archivo PDF",
            file_types=[".pdf"],  # Solo acepta PDFs
        ),
        gr.Radio(
            choices=["corto", "medio", "largo"],
            label="Longitud del resumen",
            value="medio"
        )
    ],
    outputs=gr.Textbox(
        label="Resumen del PDF",
        lines=10
    ),
    title="Resumidor de PDFs con Phi-3",
    description="Cargá un archivo PDF y obtendrás un resumen automático."
)

interfaz_pdf.launch(share=True)
```

**Para PDFs muy largos (chunking):**
```python
def resumir_pdf_largo(archivo_pdf, longitud):
    texto = extraer_texto_pdf(archivo_pdf)
    palabras = texto.split()
    
    # Dividir en chunks de 1000 palabras
    chunk_size = 1000
    chunks = [palabras[i:i+chunk_size] for i in range(0, len(palabras), chunk_size)]
    
    # Resumir cada chunk
    resumenes_parciales = []
    for chunk in chunks:
        texto_chunk = " ".join(chunk)
        resumen = resumir_con_phi3(texto_chunk, longitud="corto")
        resumenes_parciales.append(resumen)
    
    # Resumir los resúmenes (resumen de segundo nivel)
    texto_resumenes = "\n".join(resumenes_parciales)
    resumen_final = resumir_con_phi3(texto_resumenes, longitud=longitud)
    
    return resumen_final
```

**Extensiones:**
- Soportar DOCX: usar biblioteca `python-docx`
- Soportar URLs: usar `requests` + `BeautifulSoup`
- Múltiples archivos: `gr.Files(file_count="multiple")`
</details>

---

**8. La aplicación funciona bien en tu computadora pero es muy lenta cuando la compartís públicamente. ¿Qué estrategias usarías para optimizarla?**

<details>
<summary>Ver respuesta</summary>

Estrategias de optimización:

**1. Caché de resultados**
```python
from functools import lru_cache
import hashlib

# Cache simple en memoria
cache_resumenes = {}

def resumir_con_cache(texto, longitud):
    # Crear hash del texto como key
    texto_hash = hashlib.md5(texto.encode()).hexdigest()
    cache_key = f"{texto_hash}_{longitud}"
    
    # Verificar si ya está en caché
    if cache_key in cache_resumenes:
        print("[CACHE HIT] Devolviendo resumen cacheado")
        return cache_resumenes[cache_key]
    
    # Si no, generar y guardar
    resumen = resumir_con_phi3(texto, longitud)
    cache_resumenes[cache_key] = resumen
    return resumen
```

**2. Modelo cuantizado (menor precisión, más rápido)**
```python
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="auto",
    torch_dtype="float16",  # En lugar de float32
    load_in_8bit=True       # Cuantización a 8-bit (requiere bitsandbytes)
)
```

**3. Límite de rate (evitar sobrecarga)**
```python
from time import time
from collections import defaultdict

# Tracking de requests por IP
request_times = defaultdict(list)
MAX_REQUESTS_PER_MINUTE = 5

def resumir_con_rate_limit(texto, longitud, request_ip):
    now = time()
    
    # Limpiar requests antiguos (> 1 minuto)
    request_times[request_ip] = [
        t for t in request_times[request_ip]
        if now - t < 60
    ]
    
    # Verificar límite
    if len(request_times[request_ip]) >= MAX_REQUESTS_PER_MINUTE:
        return "Error: Demasiados requests. Esperá 1 minuto."
    
    # Agregar request actual
    request_times[request_ip].append(now)
    
    return resumir_con_phi3(texto, longitud)
```

**4. Cola de procesamiento (batch processing)**
```python
import queue
import threading

request_queue = queue.Queue()

def procesador_batch():
    """Worker thread que procesa requests en batches"""
    while True:
        batch = []
        # Recolectar hasta 4 requests o esperar 2 segundos
        for _ in range(4):
            try:
                batch.append(request_queue.get(timeout=2))
            except queue.Empty:
                break
        
        if batch:
            # Procesar batch
            for texto, longitud, callback in batch:
                resumen = resumir_con_phi3(texto, longitud)
                callback(resumen)

# Iniciar worker thread
threading.Thread(target=procesador_batch, daemon=True).start()
```

**5. Usar servicio de hosting especializado**
- Hugging Face Spaces (gratis, optimizado para Gradio)
- Modal.com (serverless GPU)
- Replicate.com (deploy de modelos)
- RunPod.io (GPUs baratas)

**6. Indicador de progreso**
```python
def resumir_con_progreso(texto, longitud, progress=gr.Progress()):
    progress(0, desc="Iniciando...")
    
    progress(0.3, desc="Tokenizando texto...")
    # tokenización
    
    progress(0.6, desc="Generando resumen...")
    resumen = resumir_con_phi3(texto, longitud)
    
    progress(1.0, desc="Completado!")
    return resumen
```

**Priorización:**
1. Caché (fácil, alto impacto)
2. Rate limiting (previene abuso)
3. Hosting apropiado (Spaces gratis)
4. Cuantización (si la calidad sigue siendo buena)
5. Batch processing (complejo pero escalable)
</details>

---

**9. Diseñá una interfaz que permita comparar resúmenes de tres modelos diferentes (Phi-3, mT5, y GPT-2 español) side-by-side.**

<details>
<summary>Ver respuesta</summary>

```python
import gradio as gr
from transformers import pipeline

# Cargar los tres modelos
# (Asumiendo que ya tenés Phi-3 cargado)

# mT5 para sumarización
summarizer_mt5 = pipeline(
    "summarization",
    model="csebuetnlp/mT5_multilingual_XLSum"
)

# GPT-2 español para generación
generator_gpt2 = pipeline(
    "text-generation",
    model="PlanTL-GOB-ES/gpt2-base-bne"
)

def comparar_tres_modelos(texto, longitud):
    """
    Genera resúmenes con tres modelos y devuelve comparación.
    """
    if len(texto.split()) < 50:
        return "Error: Texto muy corto", "", ""
    
    # 1. Phi-3 (LLM instruction-tuned)
    try:
        resumen_phi3 = resumir_con_phi3(texto, longitud)
    except Exception as e:
        resumen_phi3 = f"Error: {str(e)}"
    
    # 2. mT5 (Modelo especializado)
    try:
        max_lengths = {"corto": 50, "medio": 100, "largo": 150}
        resumen_mt5 = summarizer_mt5(
            texto,
            max_length=max_lengths[longitud],
            min_length=20
        )[0]['summary_text']
    except Exception as e:
        resumen_mt5 = f"Error: {str(e)}"
    
    # 3. GPT-2 español (Generación con prompt)
    try:
        prompt_gpt2 = f"Resume el siguiente texto:\n{texto}\n\nResumen:"
        max_tokens = {"corto": 50, "medio": 100, "largo": 150}
        resumen_gpt2 = generator_gpt2(
            prompt_gpt2,
            max_length=len(prompt_gpt2.split()) + max_tokens[longitud],
            do_sample=False
        )[0]['generated_text']
        # Extraer solo el resumen
        resumen_gpt2 = resumen_gpt2.split("Resumen:")[-1].strip()
    except Exception as e:
        resumen_gpt2 = f"Error: {str(e)}"
    
    return resumen_phi3, resumen_mt5, resumen_gpt2

# Interfaz con múltiples outputs
interfaz_comparacion = gr.Interface(
    fn=comparar_tres_modelos,
    
    inputs=[
        gr.Textbox(
            label="Texto a resumir",
            placeholder="Pegá el texto aquí...",
            lines=10
        ),
        gr.Radio(
            choices=["corto", "medio", "largo"],
            label="Longitud",
            value="medio"
        )
    ],
    
    outputs=[
        gr.Textbox(
            label="Resumen con Phi-3 (LLM instruction-tuned)",
            lines=6
        ),
        gr.Textbox(
            label="Resumen con mT5 (Modelo especializado)",
            lines=6
        ),
        gr.Textbox(
            label="Resumen con GPT-2 español (Generación)",
            lines=6
        )
    ],
    
    title="Comparación de Modelos de Sumarización",
    
    description="""Compara resúmenes generados por tres modelos diferentes:
    
    - **Phi-3**: LLM de propósito general con instruction tuning
    - **mT5**: Modelo encoder-decoder especializado en sumarización
    - **GPT-2**: Modelo generativo entrenado en español
    
    Observá las diferencias en estilo, fidelidad y coherencia.""",
    
    examples=[
        [
            texto_ejemplo,
            "medio"
        ]
    ]
)

interfaz_comparacion.launch(share=True)
```

**Análisis para el usuario:**
Podés agregar una cuarta columna con análisis:
```python
def analizar_resumenes(texto, phi3, mt5, gpt2):
    analisis = f"""
    ANÁLISIS COMPARATIVO:
    
    Longitudes:
    - Phi-3: {len(phi3.split())} palabras
    - mT5: {len(mt5.split())} palabras
    - GPT-2: {len(gpt2.split())} palabras
    
    Características:
    - Phi-3: Tiende a ser más estructurado y formal
    - mT5: Más conciso, a veces telegráfico
    - GPT-2: Puede ser más creativo pero menos fiel
    """
    return analisis
```
</details>

---

**10. ¿Cómo implementarías un sistema que permita a los usuarios dar feedback sobre la calidad de los resúmenes generados?**

<details>
<summary>Ver respuesta</summary>

Sistema de feedback con almacenamiento y análisis:

```python
import gradio as gr
import json
from datetime import datetime
import os

# Archivo para almacenar feedback
FEEDBACK_FILE = "feedback_resumenes.json"

def guardar_feedback(texto_original, resumen, calificacion, comentario=""):
    """
    Guarda feedback del usuario.
    """
    feedback_entry = {
        "timestamp": datetime.now().isoformat(),
        "texto_original": texto_original[:200],  # Solo primeros 200 chars
        "resumen": resumen,
        "calificacion": calificacion,  # 1-5 estrellas
        "comentario": comentario
    }
    
    # Cargar feedback existente
    if os.path.exists(FEEDBACK_FILE):
        with open(FEEDBACK_FILE, 'r') as f:
            feedback_list = json.load(f)
    else:
        feedback_list = []
    
    # Agregar nuevo feedback
    feedback_list.append(feedback_entry)
    
    # Guardar
    with open(FEEDBACK_FILE, 'w') as f:
        json.dump(feedback_list, f, indent=2)
    
    return "¡Gracias por tu feedback! Esto nos ayuda a mejorar."

# Variables globales para almacenar el último resumen
ultimo_texto = ""
ultimo_resumen = ""

def resumir_y_guardar(texto, longitud):
    """Genera resumen y guarda para feedback posterior"""
    global ultimo_texto, ultimo_resumen
    
    ultimo_texto = texto
    ultimo_resumen = resumir_con_phi3(texto, longitud)
    
    return ultimo_resumen

def procesar_feedback(calificacion, comentario):
    """Procesa el feedback del usuario"""
    global ultimo_texto, ultimo_resumen
    
    if not ultimo_resumen:
        return "Error: Primero generá un resumen"
    
    return guardar_feedback(ultimo_texto, ultimo_resumen, calificacion, comentario)

# Interfaz con dos pestañas
with gr.Blocks() as demo:
    gr.Markdown("# Resumidor con Sistema de Feedback")
    
    with gr.Tab("Resumir"):
        with gr.Row():
            with gr.Column():
                input_texto = gr.Textbox(
                    label="Texto a resumir",
                    lines=10,
                    placeholder="Pegá el texto aquí..."
                )
                input_longitud = gr.Radio(
                    choices=["corto", "medio", "largo"],
                    label="Longitud",
                    value="medio"
                )
                btn_resumir = gr.Button("Generar Resumen", variant="primary")
            
            with gr.Column():
                output_resumen = gr.Textbox(
                    label="Resumen generado",
                    lines=10
                )
        
        btn_resumir.click(
            fn=resumir_y_guardar,
            inputs=[input_texto, input_longitud],
            outputs=output_resumen
        )
    
    with gr.Tab("Dar Feedback"):
        gr.Markdown("""### ¿Qué te pareció el resumen?
        
        Tu feedback nos ayuda a mejorar el sistema.""")
        
        calificacion = gr.Slider(
            minimum=1,
            maximum=5,
            step=1,
            label="Calificación (1-5 estrellas)",
            value=3
        )
        
        comentario = gr.Textbox(
            label="Comentarios (opcional)",
            placeholder="¿Qué mejorarías? ¿Qué te gustó?",
            lines=4
        )
        
        btn_feedback = gr.Button("Enviar Feedback", variant="secondary")
        
        output_feedback = gr.Textbox(
            label="Resultado",
            lines=2
        )
        
        btn_feedback.click(
            fn=procesar_feedback,
            inputs=[calificacion, comentario],
            outputs=output_feedback
        )
    
    with gr.Tab("Estadísticas"):
        gr.Markdown("### Análisis de Feedback")
        
        def mostrar_estadisticas():
            if not os.path.exists(FEEDBACK_FILE):
                return "No hay feedback todavía"
            
            with open(FEEDBACK_FILE, 'r') as f:
                feedback_list = json.load(f)
            
            if not feedback_list:
                return "No hay feedback todavía"
            
            # Calcular estadísticas
            total = len(feedback_list)
            promedio = sum(f['calificacion'] for f in feedback_list) / total
            
            # Distribución
            distribucion = {i: 0 for i in range(1, 6)}
            for f in feedback_list:
                distribucion[f['calificacion']] += 1
            
            stats = f"""
            Total de feedback: {total}
            Calificación promedio: {promedio:.2f} / 5
            
            Distribución:
            5 estrellas: {distribucion[5]} ({distribucion[5]/total*100:.1f}%)
            4 estrellas: {distribucion[4]} ({distribucion[4]/total*100:.1f}%)
            3 estrellas: {distribucion[3]} ({distribucion[3]/total*100:.1f}%)
            2 estrellas: {distribucion[2]} ({distribucion[2]/total*100:.1f}%)
            1 estrella: {distribucion[1]} ({distribucion[1]/total*100:.1f}%)
            """
            return stats
        
        btn_stats = gr.Button("Actualizar Estadísticas")
        output_stats = gr.Textbox(label="Estadísticas", lines=10)
        
        btn_stats.click(
            fn=mostrar_estadisticas,
            inputs=[],
            outputs=output_stats
        )

demo.launch(share=True)
```

**Mejoras adicionales:**
- Almacenar en base de datos (SQLite, PostgreSQL)
- Dashboard interactivo con gráficos (plotly)
- Análisis de sentimiento en comentarios
- Identificar patrones en resúmenes mal calificados
- A/B testing de diferentes prompts basándose en feedback
</details>

---

### Desafíos Adicionales

**Desafío 1**: Implementá una función que automáticamente detecte el idioma del texto de entrada y use el modelo de sumarización más apropiado.

**Desafío 2**: Creá una interfaz que permita resumir contenido de URLs (extraer texto de páginas web y resumirlo).

**Desafío 3**: Diseñá un sistema que genere resúmenes de diferentes niveles técnicos ("para expertos", "para público general", "para niños").

---

## Referencias y Recursos Adicionales

- [Gradio Documentation](https://gradio.app/docs/)
- [Phi-3 Model Card](https://huggingface.co/microsoft/Phi-3-mini-4k-instruct)
- [Gradio Guides & Tutorials](https://gradio.app/guides/)
- [Hugging Face Spaces](https://huggingface.co/spaces) - Deploy gratuito
- [Prompt Engineering Guide](https://www.promptingguide.ai/)
- [Building ML Apps with Gradio](https://www.deeplearning.ai/short-courses/building-generative-ai-applications-with-gradio/)